In [1]:
import pandas as pd 
data = pd.read_csv('relation_new.csv')

In [3]:
set(data.关系)

{'一级科目到二级科目', '主科目到一级科目', '提问->答案', '知识点->提问'}

In [7]:
question = list(data[data['关系']=='提问->答案']['实体1'])
answer = list(data[data['关系']=='提问->答案']['实体2'])

In [11]:
import jieba
q_cut = [jieba.lcut_for_search(i) for i in question]
a_cut = [jieba.lcut_for_search(i) for i in answer]

In [17]:
from collections import defaultdict
q_count = defaultdict(int)
for i in q_cut:
    for j in i:
        q_count[j] += 1
        
a_count = defaultdict(int)
for i in a_cut:
    for j in i:
        a_count[j] += 1

In [19]:
with open('q.txt','w') as f:
    q = sorted(q_count.items(), key=lambda q_count:q_count[1], reverse=True)
    for i in q:
        f.write(i[0]+'\n')

In [21]:
with open('a.txt','w',encoding = 'utf-8') as f:
    a = sorted(a_count.items(), key=lambda a_count:a_count[1], reverse=True)
    for i in a:
        f.write(i[0]+'\n')

In [32]:
clean_word = [i.strip() for i in open('clean.txt','r',encoding = 'utf-8').readlines()]
def clean(ins):
    if '006y8m' not in ins:
        for i in clean_word:
            ins = ins.replace(i,'')
    else:
        for i in clean_word:
            if i == '006y8m':
                continue
            ins = ins.replace(i,'')
        tmp = [i for i in jieba.lcut_for_search(ins) if '006y8m' in i]
        for i in tmp:
            ins = ins.replace(i,'')
    return ins

data['实体2'] = data['实体2'].apply(clean)
data['实体1'] = data['实体1'].apply(clean)

In [36]:
# data[data['关系']=='提问->答案']

In [51]:
question = list(data[data['关系']=='提问->答案'].实体1)
answer = list(data[data['关系']=='提问->答案'].实体2)
src = data[data['关系']=='提问->答案']
src.to_csv('new.csv',index = None)

In [53]:
HanLP.segment(question[0])

In [73]:
def get_ner(in_text):
    # 创建关键词和对应标签的记录器
    segs = HanLP.segment(in_text)
    text = []
    label = []
    for ind,seg in enumerate(segs):
        if str(seg.nature) == 'speacial':
            tmp = list(seg.word)
            text += tmp
            if len(tmp) == 1:
                label += ['S-Specialty']
            if len(tmp) == 2:
                label += ['B-Specialty']
                label += ['E-Specialty']
            if len(tmp) > 2:
                label += ['B-Specialty']
                label += ['I-Specialty']*(len(tmp)-2)
                label += ['E-Specialty']
        else:
            text += list(seg.word)
            label += ['O']*len(seg.word)
    return text,label

In [19]:
import os
path = 'data/'
data = {}
for dir_0 in os.listdir(path):
    data[dir_0] = {}
    for dir_1 in os.listdir(path+dir_0) :
        data[dir_0][dir_1.split('.')[0]] = {}
        if dir_1.endswith('md'):
            data[dir_0][dir_1.split('.')[0]] = []
            f = open(path+dir_0+'/'+dir_1,'r',encoding = 'utf-8').readlines()
            q,a = [],[]
            a_keys = False
            for context in f:
                if context[0] == '#':
                    q.append(context[2:].strip())
                    a_keys = True
                    continue
                if a_keys:
                    if context == '\n':
                        a_keys = False
                        continue
                    else:
                        if len(a) < len(q):
                            a.append(context)
                        else:
                            a[-1] += context
            if len(q) == len(a):
                for i in range(len(q)):
                    data[dir_0][dir_1.split('.')[0]].append([q[i],a[i]])
            else:
                print('error',path+dir_0+'/'+dir_1)
        else:
            for dir_2 in os.listdir(path+dir_0+'/'+dir_1) :
                if dir_2.endswith('md'):
                    data[dir_0][dir_1.split('.')[0]][dir_2.split('.')[0]] = []
                    f = open(path+dir_0+'/'+dir_1+'/'+dir_2,'r',encoding = 'utf-8').readlines()
                    q,a = [],[]
                    a_keys = False
                    
                    for context in f:
                        if context[0] == '#':
                            q.append(context[2:].strip())
                            a_keys = True
                            continue
                        if a_keys:
                            if context == '\n':
                                a_keys = False
                                continue
                            else:
                                if len(a) < len(q):
                                    a.append(context)
                                else:
                                    a[-1] += context
                    if len(q) == len(a):
                        for i in range(len(q)):
                            data[dir_0][dir_1.split('.')[0]][dir_2.split('.')[0]].append([q[i],a[i]])
                    else:
                        print('error',path+dir_0+'/'+dir_1+'/'+dir_2)
                else:
                    print(path+dir_0+'/'+dir_1+'/'+dir_2)

In [1]:
# load special
from pyhanlp import *
with open('tmp/q.txt','r') as f:
    for i in f.readlines():
        i = i.strip()
        CustomDictionary.add(i,'speacial')

with open('tmp/a.txt','r',encoding = 'utf-8') as f:
    for i in f.readlines():
        i = i.strip()
        CustomDictionary.add(i,'speacial')

In [20]:
def get_ner(in_text,name):
    # 创建关键词和对应标签的记录器
    segs = HanLP.segment(in_text)
    text = []
    label = []
    for ind,seg in enumerate(segs):
        if str(seg.nature) == 'speacial':
            tmp = list(seg.word)
            text += tmp
            if len(tmp) == 1:
                label += ['S-'+name]
            if len(tmp) == 2:
                label += ['B-'+name]
                label += ['E-'+name]
            if len(tmp) > 2:
                label += ['B-'+name]
                label += ['I-'+name]*(len(tmp)-2)
                label += ['E-'+name]
        else:
            text += list(seg.word)
            label += ['O']*len(seg.word)
    return text,label

In [27]:
f = open('tagging.txt','w',encoding = 'utf-8')
for name in data:
    for i in data[name]:
        for j in data[name][i]:
            q,a = j
            text,label = get_ner(q,name)
            for ind,tmp in enumerate(text):
                if tmp in ['\n',' '] or '\\' in tmp:
                    continue
                f.write(tmp+' '+label[ind]+'\n')
            text,label = get_ner(a,name)
            for ind,tmp in enumerate(text):
                if tmp in ['\n',' '] or '\\' in tmp:
                    continue
                f.write(tmp+' '+label[ind]+'\n')
f.close()

In [18]:
name,i

('机器学习', '集成学习')

In [ ]:
with open('tagging.txt','w',encoding = 'utf-8') as f:
    for i in question:
        text,label = get_ner(i)
        for ind,i in enumerate(text):
            f.write(i+' '+label[ind]+'\n')
            
    for i in answer:
        text,label = get_ner(i)
        for ind,i in enumerate(text):
            f.write(i+' '+label[ind]+'\n')

In [84]:
import pandas as pd
data = pd.read_csv('tmp/relation_new.csv')

# {'一级科目到二级科目', '主科目到一级科目', '提问->答案', '知识点->提问'}
def namechange(ins):
    if ins == '主科目到一级科目':
        return 'root2first'
    if ins == '一级科目到二级科目':
        return 'first2second'
    if ins == '知识点->提问':
        return 'second2question'

data_left = data[data['关系'].isin(['主科目到一级科目','一级科目到二级科目','知识点->提问'])]
data_left['关系'] = data_left['关系'].apply(namechange)

D:\Anaconda3-5.3\envs\nlp\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [86]:
data_left

,实体1,实体2,关系
0,基础概念,AutoML,root2first
1,AutoML,AutoML问题构成?,second2question
3,AutoML,特征工程选择思路？,second2question
5,AutoML,模型相关的选择思路?,second2question
7,AutoML,常见优化算法思路？,second2question
...,...,...,...
656,Word2Vec,负采样流程？,second2question
658,Word2Vec,word2vec两种方法各自的优势?,second2question
660,Word2Vec,怎么衡量学到的embedding的好坏?,second2question
662,Word2Vec,word2vec和glove区别？,second2question


In [42]:
data_new = data[data['关系'] == '提问->答案']
data_new_q = list(data_new.实体1)
data_new_a = list(data_new.实体2)

In [72]:
def get_f(in_text):
    segs = HanLP.segment(in_text)
    text = []
    for ind,seg in enumerate(segs):
        if str(seg.nature) == 'speacial':
            text.append(seg.word)
    return text

data_new_f2q = []
for i in data_new_q:
    f = get_f(i)
    for f_s in f:
        data_new_f2q.append([i,f_s])
        
data_new_a2f = []
for answer in data_new_a:
    for answer_single in answer.split('\n'):
        answer_single = answer_single.replace(' ','').replace('-','')
        f = get_f(answer_single)
        for f_s in f:
            data_new_a2f.append([answer_single,f_s])
            
data_new_q2a = []
for ind,answer in enumerate(data_new_a):
    for answer_single in answer.split('\n'):
        answer_single = answer_single.replace(' ','').replace('-','')
        if answer_single == '':
            continue
        data_new_q2a.append([data_new_q[ind],answer_single])

In [106]:
q2a = pd.DataFrame(data_new_q2a)
q2a.columns = ['实体1','实体2']
q2a['关系'] = 'question2answer'

question2feature = pd.DataFrame(data_new_f2q)
question2feature.columns = ['实体1','实体2']
question2feature['关系'] = 'question2feature'

answer2feature = pd.DataFrame(data_new_a2f)
answer2feature.columns = ['实体1','实体2']
answer2feature['关系'] = 'answer2feature'

In [112]:
len(q2a),len(question2feature),len(answer2feature),len(data_left)

(1390, 421, 2099, 355)

In [116]:
new = q2a.append(question2feature).append(answer2feature).append(data_left)

In [118]:
new.to_csv('data_src.csv',index = None)

In [125]:
# {'answer2feature',
#  'first2second',
#  'question2answer',
#  'question2feature',
#  'root2first',
#  'second2question'}
entity = []
for i in range(len(new)):
    i = new.iloc[i]
    e1 = i.实体1
    e2 = i.实体2
    r = i.关系.split('2')
    entity.append([hash(e1),e1,r[0]])
    entity.append([hash(e2),e2,r[1]])    

In [131]:
entity = pd.DataFrame(entity)
entity.columns = [':ID','name',':LABEL']
entity = entity.drop_duplicates(subset=['name'],keep='first')
entity.to_csv('entity.csv',index = None)

In [148]:
def do(ins):
    return hash(ins)
new = pd.read_csv('data_src.csv')
new['name'] = new['关系']
new.columns = [':START_ID',':END_ID',':TYPE','name']
new = new[[':START_ID','name',':END_ID',':TYPE']]
new[':START_ID'] = new[':START_ID'].apply(do)
new[':END_ID'] = new[':END_ID'].apply(do)

           实体1                          实体2               关系
0  AutoML问题构成?                         特征选择  question2answer
1  AutoML问题构成?                         模型选择  question2answer
2  AutoML问题构成?                         算法选择  question2answer
3    特征工程选择思路？                     有监督的特征选择  question2answer
4    特征工程选择思路？  基于模型，lr的系数，树模型的importance等等  question2answer
5    特征工程选择思路？                  基于选择，前项后项选择  question2answer
6    特征工程选择思路？                     无监督的特征选择  question2answer
7    特征工程选择思路？           基于统计信息的，熵、相关性、KL系数  question2answer
8    特征工程选择思路？      基于方差，因子分解，PCA主成分分享，方差系数  question2answer
9   模型相关的选择思路?                         模型选择  question2answer


In [153]:
new.to_csv('relation.csv',index = False)